# Agentic AI

code snippets from this section

In [7]:
#%pip install --upgrade --quiet pydantic-ai-slim[anthropic] anthropic

In [8]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

MODEL_ID="claude-sonnet-4-20250514"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"

In [9]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()

## PydanticAI

In [10]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    name: str
    quantity_on_hand: int
    weekly_quantity_sold_past_n_weeks: [int]
    weeks_to_deliver: int

@dataclass
class Reorder:
    name: str
    quantity_to_order: int
    reason_to_reorder: str

items = [
    InventoryItem("itemA", 300, [50, 70, 80, 100], 2),
    InventoryItem("itemB", 100, [70, 80, 90, 70], 2),
    InventoryItem("itemC", 200, [80, 70, 90, 80], 1)
]

In [11]:
print("Current Inventory:")
for item in items:
    print(f"- {item.name}: {item.quantity_on_hand} units on hand")  

Current Inventory:
- itemA: 300 units on hand
- itemB: 100 units on hand
- itemC: 200 units on hand


In [12]:
from pydantic_ai import Agent

agent = Agent(
    f"anthropic:{MODEL_ID}",
    instructions="You are an inventory manager who orders just in time.",
    output_type=list[Reorder]
)

result = agent.run_sync(f"""
Identify which of these items need to be reordered this week.
Items {items}""")

print("\nReorder Recommendations:")
for reorder in result.output:
    print(f"-- Reorder {reorder.quantity_to_order} units of {reorder.name}: {reorder.reason_to_reorder}")
    


Reorder Recommendations:
-- Reorder 150 units of itemA: Current stock of 300 will last approximately 2.4 weeks based on recent average weekly sales of 75 units. With 2-week delivery time, need to reorder now to avoid stockout.
-- Reorder 240 units of itemB: Current stock of 100 will last only 1.3 weeks based on recent average weekly sales of 77.5 units. With 2-week delivery time, critically low inventory requiring immediate reorder.
